# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

### Описание данных
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

Подключение библиотек.

In [ ]:
tariff_data = pd.read_csv('/datasets/users_behavior.csv')
tariff_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [ ]:
tariff_data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Открытие файла и отображение информации о датафрейме.

In [ ]:
tariff_data.describe().T

,count,mean,std,min,25%,50%,75%,max
calls,3214.0,63.038892,33.236368,0.0,40.0000,62.000,82.0000,244.00
minutes,3214.0,438.208787,234.569872,0.0,274.5750,430.600,571.9275,1632.06
messages,3214.0,38.281269,36.148326,0.0,9.0000,30.000,57.0000,224.00
mb_used,3214.0,17207.673836,7570.968246,0.0,12491.9025,16943.235,21424.7000,49745.73
is_ultra,3214.0,0.306472,0.461100,0.0,0.0000,0.000,1.0000,1.00


Вывод сводной таблицы по датафрейму в транспорированном виде.
Данных для анализа не много, но достаточно для текущей задачи в точность >= 0.75.

### Вывод.
У нас задача классификации. 
* Целевой признак берем столбец is_ultra. 
* Остальные признаки помогут нам предсказывать решение по целевому признаку is_ultra.

Данных (125.7 KB) не очень много.

## Разбейте данные на выборки

In [ ]:
train_tariff_data, vnt_tariff_data = train_test_split(tariff_data, test_size=0.40, random_state=18032022)
valid_tariff_data, test_tariff_data = train_test_split(vnt_tariff_data, test_size=0.50, random_state=18032022)

print('Размер тренериующей выборки', train_tariff_data.shape[0])
print('Размер валидационной выборки', valid_tariff_data.shape[0])
print('Размер тестовой выборки', test_tariff_data.shape[0])

Размер тренериующей выборки 1928
Размер валидационной выборки 643
Размер тестовой выборки 643


Данные разбиваются на 60% 20 % 20 %.

In [ ]:
train_tariff_data_features = train_tariff_data.drop(['is_ultra'], axis=1)
train_tariff_data_target = train_tariff_data['is_ultra']
valid_tariff_data_features = valid_tariff_data.drop(['is_ultra'], axis=1)
valid_tariff_data_target = valid_tariff_data['is_ultra']
test_tariff_data_features = test_tariff_data.drop(['is_ultra'], axis=1)
test_tariff_data_target = test_tariff_data['is_ultra']

Создаем таблицы признаков и целового признака для каждого дата фрейма без is_ultra.

### Вывод
Разбили данные на 60% 20 % 20 %.
* Размер тренериующей выборки 1928
* Размер валидационной выборки 643
* Размер тестовой выборки 643


* Обучение пройдет на данных train_tariff_data
* Валидация модели на данных valid_tariff_data
* Лучшая модель по валидации будет применена на данных test_tariff_data

## Исследуйте модели

### Начнем с Дерева решения. 
Изменим в цикле гиперпараметр max_depth.

In [ ]:
%%time

best_depth = 0
best_accuracy = 0

for depth in range(1,101,2):
    model = DecisionTreeClassifier(random_state=18032022, max_depth=depth)
    model.fit(train_tariff_data_features, train_tariff_data_target)
    valid_predictions = model.predict(valid_tariff_data_features)
    accuracy = accuracy_score(valid_tariff_data_target, valid_predictions)
    print('Глубина дерева', depth,'Точность',accuracy)
    if accuracy > best_accuracy:
        best_depth = depth
        best_accuracy = accuracy

print('Лучшая глубина дерева', best_depth,'Лучшая точность', best_accuracy) 

Глубина дерева 1 Точность 0.7651632970451011
Глубина дерева 3 Точность 0.8087091757387247
Глубина дерева 5 Точность 0.8118195956454122
Глубина дерева 7 Точность 0.8102643856920684
Глубина дерева 9 Точность 0.807153965785381
Глубина дерева 11 Точность 0.7884914463452566
Глубина дерева 13 Точность 0.776049766718507
Глубина дерева 15 Точность 0.7418351477449455
Глубина дерева 17 Точность 0.7325038880248833
Глубина дерева 19 Точность 0.7262830482115086
Глубина дерева 21 Точность 0.6951788491446346
Глубина дерева 23 Точность 0.6998444790046656
Глубина дерева 25 Точность 0.6998444790046656
Глубина дерева 27 Точность 0.6998444790046656
Глубина дерева 29 Точность 0.6998444790046656
Глубина дерева 31 Точность 0.6998444790046656
Глубина дерева 33 Точность 0.6998444790046656
Глубина дерева 35 Точность 0.6998444790046656
Глубина дерева 37 Точность 0.6998444790046656
Глубина дерева 39 Точность 0.6998444790046656
Глубина дерева 41 Точность 0.6998444790046656
Глубина дерева 43 Точность 0.699844479004

Шаг для перебора взят 2, при увеличении глубины дерева на 1, качество почти не меняется.

In [ ]:
%%time

best_tree_model = DecisionTreeClassifier(random_state=18032022, max_depth=5)
best_tree_model.fit(train_tariff_data_features, train_tariff_data_target)
valid_predictions = best_tree_model.predict(valid_tariff_data_features)
accuracy = accuracy_score(valid_tariff_data_target, valid_predictions)
print('Глубина дерева', 5,'Точность',accuracy)

Глубина дерева 5 Точность 0.8118195956454122
CPU times: user 11 ms, sys: 153 µs, total: 11.1 ms
Wall time: 9.29 ms


Лучший показатель при глубине дерева 5

In [ ]:
model = DecisionTreeClassifier(random_state=18032022, max_depth=5, criterion="entropy")
model.fit(train_tariff_data_features, train_tariff_data_target)
valid_predictions = best_tree_model.predict(valid_tariff_data_features)
accuracy = accuracy_score(valid_tariff_data_target, valid_predictions)
print('Глубина дерева', 5,'Точность',accuracy)

Глубина дерева 5 Точность 0.8118195956454122


Настроим гиперпараметр criterion с gini на entropy.

In [ ]:
model = DecisionTreeClassifier(random_state=18032022, max_depth=5, splitter='random')
model.fit(train_tariff_data_features, train_tariff_data_target)
valid_predictions = best_tree_model.predict(valid_tariff_data_features)
accuracy = accuracy_score(valid_tariff_data_target, valid_predictions)
print('Глубина дерева', 5,'Точность',accuracy)

Глубина дерева 5 Точность 0.8118195956454122


Гиперпараметр gini показывает себя лучше. Изменим splitter с best на random.

### Перейдем к оценке случайного леса. 
Изменим в цикле гиперпараметр n_estimators.

In [ ]:
%%time

best_estim = 0
best_accuracy = 0

for estim in range(1,101):
    model = RandomForestClassifier(random_state=18032022, n_estimators=estim)
    model.fit(train_tariff_data_features, train_tariff_data_target)
    valid_predictions = model.predict(valid_tariff_data_features)
    accuracy = accuracy_score(valid_tariff_data_target, valid_predictions)
    print('Количество деревьев', estim,'Точность',accuracy)
    if accuracy > best_accuracy:
        best_estim = estim
        best_accuracy = accuracy

print('Лучшая количество деревьев', best_estim,'Лучшая точность', best_accuracy)

Количество деревьев 1 Точность 0.7107309486780715
Количество деревьев 2 Точность 0.7713841368584758
Количество деревьев 3 Точность 0.7527216174183515
Количество деревьев 4 Точность 0.7978227060653188
Количество деревьев 5 Точность 0.7744945567651633
Количество деревьев 6 Точность 0.7947122861586314
Количество деревьев 7 Точность 0.7807153965785381
Количество деревьев 8 Точность 0.8102643856920684
Количество деревьев 9 Точность 0.7822706065318819
Количество деревьев 10 Точность 0.7978227060653188
Количество деревьев 11 Точность 0.7931570762052877
Количество деревьев 12 Точность 0.7916018662519441
Количество деревьев 13 Точность 0.7838258164852255
Количество деревьев 14 Точность 0.7931570762052877
Количество деревьев 15 Точность 0.7884914463452566
Количество деревьев 16 Точность 0.7947122861586314
Количество деревьев 17 Точность 0.7947122861586314
Количество деревьев 18 Точность 0.7993779160186625
Количество деревьев 19 Точность 0.7869362363919129
Количество деревьев 20 Точность 0.788491

In [ ]:
%%time

best_forest_model = RandomForestClassifier(random_state=18032022, n_estimators=68)
best_forest_model.fit(train_tariff_data_features, train_tariff_data_target)
valid_predictions = best_forest_model.predict(valid_tariff_data_features)
accuracy = accuracy_score(valid_tariff_data_target, valid_predictions)
print('Количество деревьев', 68,'Точность',accuracy)

Количество деревьев 68 Точность 0.8133748055987559
CPU times: user 271 ms, sys: 7.89 ms, total: 278 ms
Wall time: 288 ms


Лучший показатель точности при количестве деревьев 68.

In [ ]:
model = RandomForestClassifier(random_state=18032022, n_estimators=68, criterion='entropy')
model.fit(train_tariff_data_features, train_tariff_data_target)
valid_predictions = model.predict(valid_tariff_data_features)
accuracy = accuracy_score(valid_tariff_data_target, valid_predictions)
print('Количество деревьев', 68,'Точность',accuracy)

Количество деревьев 68 Точность 0.7993779160186625


Настроим гиперпараметр criterion с gini на entropy.

In [ ]:
model = RandomForestClassifier(random_state=18032022, n_estimators=68, bootstrap=False)
model.fit(train_tariff_data_features, train_tariff_data_target)
valid_predictions = model.predict(valid_tariff_data_features)
accuracy = accuracy_score(valid_tariff_data_target, valid_predictions)
print('Количество деревьев', 68,'Точность',accuracy)

Количество деревьев 68 Точность 0.7807153965785381


Гиперпараметр gini показывает себя лучше. Изменим bootstrap с True на Flase.

In [ ]:
model = RandomForestClassifier(random_state=18032022, n_estimators=68, warm_start=True)
model.fit(train_tariff_data_features, train_tariff_data_target)
valid_predictions = model.predict(valid_tariff_data_features)
accuracy = accuracy_score(valid_tariff_data_target, valid_predictions)
print('Количество деревьев', 68,'Точность',accuracy)

Количество деревьев 68 Точность 0.8133748055987559


bootstrap с True показывает себя лучше. Изменим warm_start с False на True.

### Перейдем к логистической регрессии.
Изменим гипер параметр max_iter в цикле.

In [ ]:
%%time

best_max_iter = 0
best_accuracy = 0

for max_iter in range(1,201):
    model = LogisticRegression(random_state=18032022, max_iter=max_iter)
    model.fit(train_tariff_data_features, train_tariff_data_target)
    valid_predictions = model.predict(valid_tariff_data_features)
    accuracy = accuracy_score(valid_tariff_data_target, valid_predictions)
    print('Количество итерраций', max_iter,'Точность',accuracy)
    if accuracy > best_accuracy:
        best_max_iter = max_iter
        best_accuracy = accuracy

print('Лучшая количество итерраций', best_max_iter,'Лучшая точность', best_accuracy)    

Количество итерраций 1 Точность 0.7216174183514774
Количество итерраций 2 Точность 0.7216174183514774
Количество итерраций 3 Точность 0.7216174183514774
Количество итерраций 4 Точность 0.7216174183514774
Количество итерраций 5 Точность 0.7216174183514774
Количество итерраций 6 Точность 0.7216174183514774
Количество итерраций 7 Точность 0.7216174183514774
Количество итерраций 8 Точность 0.7216174183514774
Количество итерраций 9 Точность 0.7216174183514774
Количество итерраций 10 Точность 0.7216174183514774
Количество итерраций 11 Точность 0.7216174183514774
Количество итерраций 12 Точность 0.7216174183514774
Количество итерраций 13 Точность 0.7216174183514774
Количество итерраций 14 Точность 0.7216174183514774
Количество итерраций 15 Точность 0.7216174183514774
Количество итерраций 16 Точность 0.7216174183514774
Количество итерраций 17 Точность 0.7216174183514774
Количество итерраций 18 Точность 0.7216174183514774
Количество итерраций 19 Точность 0.7216174183514774
Количество итерраций 

In [ ]:
%%time

best_reg_model = LogisticRegression(random_state=18032022)
best_reg_model.fit(train_tariff_data_features, train_tariff_data_target)
valid_predictions = best_reg_model.predict(valid_tariff_data_features)
accuracy = accuracy_score(valid_tariff_data_target, valid_predictions)
print('Количество итерраций', 100,'Точность',accuracy)

Количество итерраций 100 Точность 0.7620528771384136
CPU times: user 31.2 ms, sys: 4.11 ms, total: 35.3 ms
Wall time: 45.1 ms


In [ ]:
%%time

best_solver = ''
best_accuracy = 0

for solve in {'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'}:
    model = LogisticRegression(random_state=18032022, solver=solve)
    model.fit(train_tariff_data_features, train_tariff_data_target)
    valid_predictions = model.predict(valid_tariff_data_features)
    accuracy = accuracy_score(valid_tariff_data_target, valid_predictions)
    print('Solver', solve,'Точность',accuracy)
    if accuracy > best_accuracy:
        best_solver= solve
        best_accuracy = accuracy

print('Лучший solver', best_solver,'Лучшая точность', best_accuracy)

Solver newton-cg Точность 0.7620528771384136
Solver sag Точность 0.7216174183514774
Solver saga Точность 0.7216174183514774
Solver lbfgs Точность 0.7620528771384136
Solver liblinear Точность 0.7651632970451011
Лучший solver liblinear Лучшая точность 0.7651632970451011
CPU times: user 159 ms, sys: 8 ms, total: 167 ms
Wall time: 175 ms


Значение по-умолчанию в max_iter=100 хорошее. Как и у 97, 98 и 99 итераций. Увеличение в большую сторону ничего не меняет. Изменим другой гиперпараметр solver.

In [ ]:
%%time

best_C = 0.0
best_accuracy = 0

for float_c in [x/10 for x in range(1,21)]:
    model = LogisticRegression(random_state=18032022, C=float_c)
    model.fit(train_tariff_data_features, train_tariff_data_target)
    valid_predictions = model.predict(valid_tariff_data_features)
    accuracy = accuracy_score(valid_tariff_data_target, valid_predictions)
    print('C=', float_c,'Точность',accuracy)
    if accuracy > best_accuracy:
        best_C= float_c
        best_accuracy = accuracy

print('Лучший C', best_C,'Лучшая точность', best_accuracy)

C= 0.1 Точность 0.7620528771384136
C= 0.2 Точность 0.7667185069984448
C= 0.3 Точность 0.7620528771384136
C= 0.4 Точность 0.7620528771384136
C= 0.5 Точность 0.7620528771384136
C= 0.6 Точность 0.7620528771384136
C= 0.7 Точность 0.7620528771384136
C= 0.8 Точность 0.7636080870917574
C= 0.9 Точность 0.7620528771384136
C= 1.0 Точность 0.7620528771384136
C= 1.1 Точность 0.7620528771384136
C= 1.2 Точность 0.7620528771384136
C= 1.3 Точность 0.7620528771384136
C= 1.4 Точность 0.7620528771384136
C= 1.5 Точность 0.7620528771384136
C= 1.6 Точность 0.7620528771384136
C= 1.7 Точность 0.7620528771384136
C= 1.8 Точность 0.7620528771384136
C= 1.9 Точность 0.7620528771384136
C= 2.0 Точность 0.7620528771384136
Лучший C 0.2 Лучшая точность 0.7667185069984448
CPU times: user 598 ms, sys: 3.97 ms, total: 602 ms
Wall time: 622 ms


Лучшим solver является lbfgs, который стоит по-умолчанию. На равне с ним newton-cg. Изменим в цикле гиперпараметр С.
Значение по умолчанию C=0.2 показывает лучший результат.

### Вывод
В ходе исследования были получены следующие данные:

* Точность модели решающего дерева Лучшая глубина дерева 5 Лучшая точность 0.8118195956454122
* Точность модели случайного леса Лучшая количество деревьев 68 Лучшая точность 0.8133748055987559
* Точность модели логистической регресии Лучшая количество итерраций 65 Лучшая точность 0.7667185069984448


Так как выборка небольшая, то результат решающего древа с гиперпараметром max_depth=5 почти лучший.

## Проверьте модель на тестовой выборке

In [ ]:
test_predictions = best_tree_model.predict(test_tariff_data_features)
accuracy = accuracy_score(test_tariff_data_target, test_predictions)
print('Точность модели решающего леса на тестовой выборке',accuracy)

test_predictions = best_forest_model.predict(test_tariff_data_features)
accuracy = accuracy_score(test_tariff_data_target, test_predictions)
print('Точность модели случайного леса на тестовой выборке',accuracy)

test_predictions = best_reg_model.predict(test_tariff_data_features)
accuracy = accuracy_score(test_tariff_data_target, test_predictions)
print('Точность модели логистической регресиии на тестовой выборке',accuracy)

Точность модели решающего леса на тестовой выборке 0.8040435458786936
Точность модели случайного леса на тестовой выборке 0.7900466562986003
Точность модели логистической регресиии на тестовой выборке 0.7698289269051322


Оценим точность каждой модели на тестовой выборке.

### Вывод
Для решения поставленной задачи точности 0.75 можно выбрать любую модель, лучшая будет решающим лесом

## (бонус) Проверьте модели на адекватность

Оценить адекватность модели можно используя несколько метрик:

* accuracy - точность совпадения прогнозов с правильными ответами.
* precision - отношение правильно предсказанных классов равных 1(TP) к сумме правильно предсказанных классов равных 1(TP) с неправльными предсказаниями, указанные как 1(FP).
* recall - отношение правильно предсказанных классов равных 1(TP) к сумме правильно предсказанных классов равных 1(TP) с неправльными предсказаниями, указанные как 0(FN).

In [ ]:
test_predictions = best_forest_model.predict(test_tariff_data_features)
accuracy = accuracy_score(test_tariff_data_target, test_predictions)
precision = precision_score(test_tariff_data_target, test_predictions)
recall = recall_score(test_tariff_data_target, test_predictions)
print('Accuracy =', accuracy, 'Precision =', precision, 'Recall =', recall)

Accuracy = 0.7900466562986003 Precision = 0.66 Recall = 0.5409836065573771


Для оценки адеватности используется F-мера - среднее гармоническое между precision и recall. Если хотя бы один из параметров близок к нулю, то и F-мера стремится к 0. Если оба стремятся к 1, то F-мера тоже стремится к 1.

In [ ]:
f_score = f1_score(test_tariff_data_target, test_predictions)
print('F-мера =', f_score)

F-мера = 0.5945945945945946


### Вывод
Адекватность модели оценивается по нескольким параметрам. Для модели классификации это accuracy, precision и recall. Качество модели лучше всего отражают precision и recall и эти метрики складываются в F-мера как единую оценку модели.

Модель получилась среднего качества. И для таких задач она адекватна.